# Sistemas Inteligentes 2021/2022

## Mini-projeto 1: Pacman comilão

<img src="pacman.png" alt="Drawing" style="width: 100px;"/>

## Grupo: 42

### Elementos do Grupo

Número:  53687       Nome:   Ariana Dias   
Número:  53746       Nome:   Andrei Tataru  
Número:  51127       Nome:   Luís Ferreirinha  

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação dos estados

(descreva aqui, textualmente, como decidiu representar os estados em Python; ilustre nas células de código abaixo a representação em Python de um estado à sua escolha)

Para representar-mos os estados em python, decidimos criar uma classe _EspacoPacman_, onde cada objeto desta classe representa um estado do nosso problema.
 Para este problema em concreto decidimos que estes objetos devem guardar a posição do pacman, o número de pontos acumulados, o tempo desde a última captura de uma pastilha, casas visitadas e o número de vezes que estas foram visitadas e finalmente as coordenadas e tipo das pastilhas que ainda se encontram no campo.

- Representação dos Estados:  
    - Posição Pacman: tuple $\left(x,y\right)$ onde $x,y$ são inteiros
    - Pontos: inteiro p
    - Tempo: inteiro t
    - Casas Visitadas: dicionário onde os elementos têm a seguinte forma $\{(x,y) : n\}$, $x,y,n$ são inteiros e $n$ representa o número de visitas a essa casa.
    - Conjunto das Pastilhas: dicionário onde os elementos têm a seguinte forma $\{(x,y) : tipo\}$, $x,y$ são inteiros e tipo é um character que representa o tipo da pastilha


In [2]:
# se definiu uma classe para representar os estados, inclua aqui o código Python correspondente
import copy

class EstadoPacman():
    """Objetos desta classe representam um estado do espaço do problema Pacman comilao
    """

    def __init__(self, posicao_pacman, pastilhas, pontos=0, tempo=0, visitas={}):
        self.pos = posicao_pacman
        self.pontos = pontos
        self.tempo = tempo
        self.pastilhas = copy.deepcopy(pastilhas)
        if visitas == {}:
            self.visitas = {posicao_pacman : 1}
        else:
            self.visitas = copy.deepcopy(visitas)

    def __lt__(self, other):
        if self.pontos < other.pontos:
            return True
        if self.tempo < other.tempo:
            return True
        if len(self.visitas) < len(other.visitas):
            return True
        if len(self.visitas) > len(other.visitas):
            return False
        for pos in self.visitas.keys():
            if pos in other.visitas.keys():
                if self.visitas[pos] < other.visitas[pos]:
                    return True
        return False
            
    def __eq__(self,other):
        if not isinstance(other, EstadoPacman): 
            return False
        return other.pos == self.pos and self.pontos == other.pontos \
            and self.tempo == other.tempo and self.pastilhas == other.pastilhas \
                and self.visitas == other.visitas
    
    def __hash__(self):
        return hash((self.pos, self.pontos, self.tempo)) + \
               hash(frozenset(self.pastilhas)) + \
               hash(frozenset(self.visitas))

    def __repr__(self):
        return str([self.pos, self.pontos, self.tempo, self.pastilhas, self.visitas])

    def __str__(self):
        return f"Posição Pacman: {self.pos}\nPontos: {self.pontos}\nTempo desde última pastilha: {self.tempo}\
                \nPastilhas restantes: {self.pastilhas}\nCasas Visitadas: {self.visitas}"


Vamos representar o seguinte estado:
-   Pacman está em (1,2)
-   Tem 2 pontos
-   Passaram 2 unidades de tempo desda a última pastilha comida
-   Restam as seguintes pastilhas: {(5,6) : "N", (3,2) : "D"}
-   Foram visitadas as seguintes casas: {(0,0) : 1, (1,0) : 1, (1,1) : 1, (1,2) : 1}

In [3]:
# Criamos um objecto do tipo EstadoPacman com os parametros anteriores
pastilhas_exemplo = {(5,6) : "N", (3,2) : "D"}
visitas_exemplo = {(0,0) : 1, (1,0) : 1, (1,1) : 1, (1,2) : 1}
estado_exemplo = EstadoPacman((1,2), pastilhas_exemplo , 2, 2, visitas_exemplo)

Como foi definido o método ```__str__``` para esta classe podemos imprimir o objecto para visualizar melhor o estado

In [4]:
print(estado_exemplo)

Posição Pacman: (1, 2)
Pontos: 2
Tempo desde última pastilha: 2                
Pastilhas restantes: {(5, 6): 'N', (3, 2): 'D'}
Casas Visitadas: {(0, 0): 1, (1, 0): 1, (1, 1): 1, (1, 2): 1}


## Formulação do problema

(explique textualmente como decidiu formular em Python o seu problema)

Para formular-mos este problema, começamos por definir como deve ser um estado inicial do nosso problema:

- Estado inicial:
    
    | EstadoPacman |
    | :---: |
    | Posição: $(x,y)$ |
    | Pastilhas: dict(pastilhas iniciais) |
    | Pontos: 0 |
    | Tempo: 0 |
    | Visitas: {(x,y) : 1} |

Neste estado o pacman pode começar em qualquer posição em qualquer mundo, mas o importante é que inicia com 0 pontos e com o tempo a 0. Fez-se o mesmo para o estado final.


- Estado final:

    | EstadoPacman |
    | :---: |
    | Posição: $(x_2,y_2)$ |
    | Pastilhas: dict(pastilhas restantes) |
    | Pontos: Pontos Objectivo |
    | Tempo: 0 |
    | Visitas: dict(casas visitadas n vezes) |  

Outra vez o estado pode ter qualquer configuração desde que os Pontos sejam iguais aos pontos objetivo definidos. Isto permite-nos então definir um goal test para o nosso problema.


- Goal Test: Pontos $\geq$ Pontos Objetivo

Após a execução de qualquer uma ação verifica-se se o estado é final utilizando o goal test.

Para este problema definimos duas ações principais, a primeira permite ao pacman mover-se em qualquer uma das direções ortogonais desde que não exista nenhum obstáculo a bloquear o seu caminho. A segunda vai decorrer automaticamente quando o pacman move-se para uma casa com uma pastilha, isto é ele apanha sozinho a pastilha que se encontra nessa casa.

- Operadores:  
    - Mover-se numa das quatro direcções ortogonais para as quatro células vizinhas, se não for impedido pelos limites do mundo nem por obstáculos:  
        - Norte: $(x,y) \to (x+1,y)$
        - Sul: $(x,y) \to (x-1,y)$
        - Este: $(x,y) \to (x,y+1)$
        - Oeste: $(x,y) \to (x,y-1)$
    - Se a casa onde o pacman se encontra contem uma pastilha, ele come automaticamente a pastilha, removendo esta e adicionando os respetivos pontos ao seu total de pontos:
        - Pontos Totais = Pontos Totais + Pontos da Pastilha

Mas cada ação de movimento do pacman vai ter um custo associado, visitar casas pela n-ézima vez tem um custo associado de n. Portanto é importante guardar o número de visitas a cada casa em cada estado, neste caso isto é feito com um dicionário.

- Custos:
    Custo associado a ir para a casa $(x,y)$ pela $n$-ézima vez = $n$

Sempre que o pacman apanha uma pastilha ele vai receber um número de pontos que depende do tipo da pastilha e do tempo que demorou a chegar a esta. Portanto em cada estado vamos guardar a posição de cada pastilha e o seu tipo (feito com um dicionário) e também o tempo para assim pudermos efetuar este cálculo de pontos que é feito da seguinte forma:

- Cálculo de pontos:
    - Pastilha N: Pontos += 1
    - Pastilha D: Pontos += max(0,5-tempo)
    - Pastilha C: Pontos += tempo

Nota: O descolamento de uma casa para a outra é considerado como uma iteração e portanto um acréscimo de 1 ao tempo decorrido desde a última captura de uma pastilha. Portanto quando o pacman apanha uma pastilha de qualquer tipo e imediatamente apanha uma pastilha do tipo "D" o número de pontos atribuido vai ser (max,5-1) $\to$ 4, pois ele demorou uma iteração a deslocar-se desde a casa anterior para a casa da pastilha "D" apesar de estarem uma ao lado da outra.  

Assim para cada estado existe um conjunto de ações possíveis que podem ser obtidas com o método _actions_, cada uma destas pode ser aplicada a um estado de forma a obter um estado seguinte utilizando o método _result_, sempre que se executa uma ação num estado vai ser aplicado o operador do movimento, aumenta-se o tempo em 1 e verifica-se se existem pastilhas nessa casa, se sim vai ser feito o cálculo dos pontos e adicionados ao total, a pastilha depois é removida do dicionário das pastilhas e as casas visitadas são atualiadas. Como todos estes novos parâmetros obtidos é então construido um novo estado com a classe EstadoPacman.


In [5]:
from searchPlus import *
import copy

class PacmanPastilhas(Problem):

    moves = {"N" : (0,-1), "S" : (0,1), "O" : (-1,0), "E" : (1,0)}

    def __init__(self, pos : tuple[int,int], objetivo : int, pastilhas : dict, obstaculos : set, dimensao : int):
        self.initial = EstadoPacman(pos, pastilhas)
        self.goal = objetivo
        self.obstacles = obstaculos
        self.dim = dimensao
        

    def calculate_points(self, time : int, pastilha : str) -> int:
        """Calcula os pontos a serem atribuidos por apanhar uma pastilha,
        utilizando o tempo decorrido desde a ultima pastilha capturada e 
        o tipo da pastilha

        Args:
            time (int): Tempo decorrido desde a ultima captura de uma pastilha
            pastilha (str): Tipo de pastilha

        Returns:
            int: Pontos a serem atribuidos
        """
        if pastilha == "N":
            return 1
        elif pastilha == "D":
            return max(0,5-time)
        else:
            return time
    
    def calculate_position(self, pos : tuple[int,int], direction : str) -> tuple[int,int]:
        """Dado a posicao atual do pacman e uma direcao ortogonal, devolve a posicao
        do pacman apos se mover nessa direcao"""
        return (pos[0]+self.moves[direction][0], pos[1]+self.moves[direction][1])

    def actions(self, state : EstadoPacman) -> list[str]:
        if self.goal_test(state): return []
        return [d for d in self.moves.keys() if self.calculate_position(state.pos, d) not in self.obstacles]

    def result(self, state : EstadoPacman, action : str) -> EstadoPacman:
        new_position = self.calculate_position(state.pos, action)
        new_visitas = copy.deepcopy(state.visitas)
        if new_position in new_visitas.keys():
            new_visitas[new_position] += 1
        else:
            new_visitas[new_position] = 1

        if new_position in state.pastilhas.keys():
            new_points = state.pontos + self.calculate_points(state.tempo+1, state.pastilhas[new_position])
            new_pastilhas = copy.deepcopy(state.pastilhas)
            new_pastilhas.pop(new_position)
            new_time = 0
            return EstadoPacman(new_position, new_pastilhas, new_points, new_time, new_visitas)
        
        return EstadoPacman(new_position, state.pastilhas, state.pontos, state.tempo+1, new_visitas)


    def path_cost(self, cost : int, state, action, next_state : EstadoPacman) -> int:
        return cost+next_state.visitas[next_state.pos]

    def display(self, state : EstadoPacman):
        board = ""
        for y in range(self.dim):
            for x in range(self.dim):
                if state.pos == (x,y):
                    board += "@ "
                elif (x,y) in self.obstacles:
                    board += "# "
                elif (x,y) in state.pastilhas.keys():
                    board += f"{state.pastilhas[(x,y)]} "
                else:
                    board += ". "
            board += "\n"
        print(board)
    
    def display_trace(self, plan : list[str]):
        path = set()
        state = self.initial
        for action in plan[:-1]:
            state = self.result(state, action)
            path.add(state.pos)
        
        board = ""
        for y in range(self.dim):
            for x in range(self.dim):
                if self.initial.pos == (x,y):
                    board += "@ "
                elif (x,y) in self.obstacles:
                    board += "# "
                elif (x,y) in state.pastilhas.keys():
                    board += f"{state.pastilhas[(x,y)]} "
                elif (x,y) in path:
                    board += "+ "
                else:
                    board += ". "
            board += "\n"
        print(board)

    def goal_test(self, state : EstadoPacman) -> bool:
        return state.pontos >= self.goal

## Criação de estados e do problema

(Mostrem que o código está a funcionar, construindo instâncias da classe **PacmanPastilhas**, fazendo display dos estados, verificando o teste do estado final, gerando as ações para alguns estados, executando ações a partir de alguns estados e gerando novos estados e mostrando a evolução dos custos; verificando que os estados não se modificam com as ações (são gerados novos estados) e que a igualdade e a comparação entre estados funciona. Mostrem que a execução de sequências de ações está a funcionar bem.)

In [6]:
# Funcoes auxiliares

def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)

def exec(p,estado,accoes):
    custo = 0
    for a in accoes:
        seg = p.result(estado,a)
        custo = p.path_cost(custo,estado,a,seg)
        estado = seg
    p.display(estado)
    print('Custo:',custo)
    print('Goal?',p.goal_test(estado))
    return (estado,custo)

Para demonstrar o funcionamento do nosso código vamos definir duas instâncias de PacmanPastilhas para demonstrar as funcionalidades da classe.
   
- Primeiro vamos definir um conjunto de obstaculos e das pastilhas para o primeiro mundo

In [7]:
# Mundo 1 - dimensao 10
fronteira1 = quadro(0,0,10)
obs1_1 = line(2,2,1,0,5)
obs1_2 = line(7,7,0,-1,6)
mundo_1 = fronteira1 | obs1_1 | obs1_2

pastilhas_1 = {(5,5) : "C", (4,7) : "N", (1,3) : "D"}

Vamos iniciar o estado com o pacman na posição $(1,1)$ e com objetivo 5 pontos

In [8]:
p1 = PacmanPastilhas((1,1),5,pastilhas_1,mundo_1,10) # Mundo 1

Vamos imprir o estado inicial do problema

In [9]:
# Representacao dos parametros do estado
print(p1.initial)
print() # linha vazia para melhorar visibilidade
# Representacao do estado
p1.display(p1.initial)

Posição Pacman: (1, 1)
Pontos: 0
Tempo desde última pastilha: 0                
Pastilhas restantes: {(5, 5): 'C', (4, 7): 'N', (1, 3): 'D'}
Casas Visitadas: {(1, 1): 1}

# # # # # # # # # # 
# @ . . . . . . . # 
# . # # # # # # . # 
# D . . . . . # . # 
# . . . . . . # . # 
# . . . . C . # . # 
# . . . . . . # . # 
# . . . N . . # . # 
# . . . . . . . . # 
# # # # # # # # # # 



Podemos também verificar que o estado inicial não é solução do problema

In [10]:
print(p1.goal_test(p1.initial))

False


Vamos agora ver as ações disponíveis e aplicar algumas ao nosso problema de forma a gerar uma pequena sequência de estados

- Ações disponíveis a partir do estado inicial:

In [11]:
print(p1.actions(p1.initial))

['S', 'E']


- Vamos aplicar a ação "S" (mover uma casa para baixo), ver o estado resultante utilizando p1.display(estado) e vamos também guardar o custo

In [12]:
p1_s2 = p1.result(p1.initial, "S")
p1.display(p1_s2)
custo = p1.path_cost(0, p1.initial, "S", p1_s2)
print(f"custo: {custo}")
print(f"\nAções:\n {p1.actions(p1_s2)}")

# # # # # # # # # # 
# . . . . . . . . # 
# @ # # # # # # . # 
# D . . . . . # . # 
# . . . . . . # . # 
# . . . . C . # . # 
# . . . . . . # . # 
# . . . N . . # . # 
# . . . . . . . . # 
# # # # # # # # # # 

custo: 1

Ações:
 ['N', 'S']


- Vamos agora mover o pacman outra casa para baixo de forma a capturar a pastilha do tipo D

In [13]:
p1_s3 = p1.result(p1_s2, "S")
p1.display(p1_s3)
custo = p1.path_cost(custo, p1_s2, "S", p1_s3)
print(f"custo: {custo}")
print(f"\nAções:\n {p1.actions(p1_s3)}")

# # # # # # # # # # 
# . . . . . . . . # 
# . # # # # # # . # 
# @ . . . . . # . # 
# . . . . . . # . # 
# . . . . C . # . # 
# . . . . . . # . # 
# . . . N . . # . # 
# . . . . . . . . # 
# # # # # # # # # # 

custo: 2

Ações:
 ['N', 'S', 'E']


- Se voltarmos uma casa atrás o custo vai acrescentar 2 porque ja visitamos aquela casa uma vez antes

In [14]:
cost_test1 = p1.result(p1_s3, "N")
custo2 = p1.path_cost(custo, p1_s3, "N", cost_test1)
print(f"custo: {custo2}")

custo: 4


- Podemos ver o número de pontos que o pacman tem ao fazer print do estado com print(estado), como os estados não são modificados após a aplicação de uma ação podemos visualizar e comparar estados anteriores

In [15]:
print("Segundo estado (p1_s2):")
print(p1_s2)
print()
print("Terceiro estado (p1_s3):")
print(p1_s3)

print("\nComparações:")
print("p1_s2 == p1_s3 ?")
print(p1_s2 == p1_s3)
print("p1_s2 < p1_s3 ?")
print(p1_s2 < p1_s3)
print("\n p1_s2 tem menos pontos do que p1_s3 logo é menor")


Segundo estado (p1_s2):
Posição Pacman: (1, 2)
Pontos: 0
Tempo desde última pastilha: 1                
Pastilhas restantes: {(5, 5): 'C', (4, 7): 'N', (1, 3): 'D'}
Casas Visitadas: {(1, 1): 1, (1, 2): 1}

Terceiro estado (p1_s3):
Posição Pacman: (1, 3)
Pontos: 3
Tempo desde última pastilha: 0                
Pastilhas restantes: {(5, 5): 'C', (4, 7): 'N'}
Casas Visitadas: {(1, 1): 1, (1, 2): 1, (1, 3): 1}

Comparações:
p1_s2 == p1_s3 ?
False
p1_s2 < p1_s3 ?
True

 p1_s2 tem menos pontos do que p1_s3 logo é menor


- Podemos também criar uma instância de EstadoPacman com os mesmos atributos que p1_s2 e mostrar que são iguais

In [16]:
eg1 = EstadoPacman(p1_s2.pos, p1_s2.pastilhas, p1_s2.pontos, p1_s2.tempo, p1_s2.visitas)
print(eg1.pastilhas == p1_s2.pastilhas)

True


- Vamos agora aplicar uma sequência de ações que nos faz chegar ao estado final

In [17]:
# Seq
actions = ["S"]*2 + ["E"]*4 + ["S"]*2
p1_final = exec(p1, p1.initial, actions)
p1.actions(p1_final[0])

# # # # # # # # # # 
# . . . . . . . . # 
# . # # # # # # . # 
# . . . . . . # . # 
# . . . . . . # . # 
# . . . . @ . # . # 
# . . . . . . # . # 
# . . . N . . # . # 
# . . . . . . . . # 
# # # # # # # # # # 

Custo: 8
Goal? True


[]

Finalmente, vamos criar uma segunda instância de PacmanPastilhas para demonstrar um problema com um mundo de dimensão diferente e com obstáculos diferentes.

In [18]:
fronteira2 = quadro(0,0,15)
obs2_1 = line(2,1,0,1,12)
obs2_2 = line(2,12,1,0,11)
obs2_3 = line(5,3,0,1,9)
obs2_4 = line(10,1,0,1,10)
mundo_2 = fronteira2 | obs2_1 | obs2_2 | obs2_3 | obs2_4

pastilhas_2 = {(3,4) : "C", (12,1) : "D", (1,3) : "D", (1,13) : "N", (12,3) : "N"}

p2 = PacmanPastilhas((1,1),8,pastilhas_2,mundo_2,15) # Mundo 2

# Representacao dos parametros do estado
print(p2.initial)
print()
# Representacao do estado
p2.display(p2.initial)

Posição Pacman: (1, 1)
Pontos: 0
Tempo desde última pastilha: 0                
Pastilhas restantes: {(3, 4): 'C', (12, 1): 'D', (1, 3): 'D', (1, 13): 'N', (12, 3): 'N'}
Casas Visitadas: {(1, 1): 1}

# # # # # # # # # # # # # # # 
# @ # . . . . . . . # . D . # 
# . # . . . . . . . # . . . # 
# D # . . # . . . . # . N . # 
# . # C . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . . . . . # 
# . # # # # # # # # # # # . # 
# N . . . . . . . . . . . . # 
# # # # # # # # # # # # # # # 



In [19]:
p2_solution = ["S"]*12+["E"]*12+["N"]*10+["O"]+["N"]*2
p2_final = exec(p2, p2.initial, p2_solution)
p2.display_trace(p2_solution)

# # # # # # # # # # # # # # # 
# . # . . . . . . . # . @ . # 
# . # . . . . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # C . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . . . . . # 
# . # # # # # # # # # # # . # 
# . . . . . . . . . . . . . # 
# # # # # # # # # # # # # # # 

Custo: 37
Goal? True
# # # # # # # # # # # # # # # 
# @ # . . . . . . . # . D . # 
# + # . . . . . . . # . + . # 
# + # . . # . . . . # . + + # 
# + # C . # . . . . # . . + # 
# + # . . # . . . . # . . + # 
# + # . . # . . . . # . . + # 
# + # . . # . . . . # . . + # 
# + # . . # . . . . # . . + # 
# + # . . # . . . . # . . + # 
# + # . . # . . . . # . . + # 
# + # . . # . . . . . . . + # 
# + # # # # # # # # # # # + # 
# + + + + + + + + + + + + + # 
# # # # # # # # # # # # # # # 



## Teste de procura de solução

(utilização de algoritmos de procura aprendidos nas aulas e comparação dos resultados ao nível de tempo de execução e solução obtida; comente aqui os resultados obtidos e o que observa)

Vamos agora aplicar os seguintes algoritmos de procura ao nosso problema:
- Profundidade-primeiro
- Largura-primeiro
- Aprofundamento Progressivo
- Custo Uniforme

Cada um destes algoritmos será aplicado em grafo. Iremos medir também o tempo de execução de cada um deste e no final iremos comparar todos e tirar conclusões.

In [20]:
import sys

# Algoritmo de procura em largura-primeiro em grafo

def breadth_first_graph_search(problem):
    """Search the deepest nodes in the search tree first."""
    return graph_search(problem, FIFOQueue())

# Algoritmo de procura de aprofundamento progressivo em grafo

def limited_graph_search(problem, frontier, limit):
    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        if node.depth < limit:
            explored.add(node.state)
            frontier.extend(child for child in node.expand(problem)
                            if child.state not in explored and
                            child not in frontier)
    return None
        
def progressive_depth_search_graph(problem):
        for i in range(sys.maxsize):
            result = limited_graph_search(problem, Stack(), i)
            if result is not None:
                return result
        return result

O problema que iremos utilizar será uma versão ligeiramente mais pequena do problema1 definido anteriormente.

In [21]:
fronteira3 = quadro(0,0,8)
obs3_1 = line(2,2,1,0,3)
obs3_2 = line(5,2,0,1,4)
mundo_3 = fronteira3 | obs3_1 | obs3_2

pastilhas_3 = {(6,5) : "C", (1,6) : "N", (2,6) : "D", (4,1) : "D", (2,1) : "C"}

p3 = PacmanPastilhas((1,1),5,pastilhas_3,mundo_3,8)
p3.display(p3.initial)

# # # # # # # # 
# @ C . D . . # 
# . # # # # . # 
# . . . . # . # 
# . . . . # . # 
# . . . . # C # 
# N D . . . . # 
# # # # # # # # 



- Pronfundidade-Primeiro

In [29]:
# Cuidado ciclo infinito

import timeit
start = timeit.default_timer()
resultado = depth_first_graph_search(p3)
print("Solução Profundidade-primeiro (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
p3.display_trace(resultado.solution())

KeyboardInterrupt: 

- Largura-Primeiro  

In [23]:
import timeit
start = timeit.default_timer()
resultado = breadth_first_graph_search(p3)
print("Solução Lagura-primeiro (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
p3.display_trace(resultado.solution())

Solução Lagura-primeiro (grafo) com custo 9:
['S', 'N', 'S', 'N', 'E']
Time:  0.0067479999852366745
# # # # # # # # 
# @ C . D . . # 
# + # # # # . # 
# . . . . # . # 
# . . . . # . # 
# . . . . # C # 
# N D . . . . # 
# # # # # # # # 



- Aprofundamento Progressivo

In [24]:
import timeit
start = timeit.default_timer()
resultado = progressive_depth_search_graph(p3)
print("Solução Aprofundamento Progressivo (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
p3.display_trace(resultado.solution())

Solução Aprofundamento Progressivo (grafo) com custo 7:
['S', 'S', 'N', 'N', 'E']
Time:  0.007620299991685897
# # # # # # # # 
# @ C . D . . # 
# + # # # # . # 
# + . . . # . # 
# . . . . # . # 
# . . . . # C # 
# N D . . . . # 
# # # # # # # # 



- Custo Uniforme

In [27]:
import timeit
start = timeit.default_timer()
resultado = uniform_cost_search(p3)
print("Solução Custo Uniforme (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
p3.display_trace(resultado.solution())

Solução Custo Uniforme (grafo) com custo 6:
['S', 'S', 'S', 'S', 'S', 'E']
Time:  0.01503529999172315
# # # # # # # # 
# @ C . D . . # 
# + # # # # . # 
# + . . . # . # 
# + . . . # . # 
# + . . . # C # 
# + D . . . . # 
# # # # # # # # 



- Comparação e discusão  

In [28]:
Tempos = [0.0067479999852366745, 0.007620299991685897, 0.018449399969540536]
desvios = [(x / Tempos[0])*100-100 for x in Tempos[1:]]
print(desvios)

[12.92679324774224, 173.40545361446755]


| Algoritmo | Tempo demorado (s) | custo | desvio (%) |
| :---: |:------------------:|:-----:|:----------:|
| Profundidade-Primeiro |       -----        |  --   |     --     |
| Largura-Primeiro |      0.00675       |   9   |     +0     |
| Aprofundamento Progressivo |      0.00762       |   7   |    +13     |
| Custo Uniforme |      0.01504       |   6   |    +173    |

<br />

De acordo com os resultados obtidos o algoritmo mais rápido para esta configuração em específico do nosso problema foi a procura em Largura-Primeiro em grafo, todos os outros algoritmos demoram mais. Apesar de ser o mais rápido não é obteve a melhor solução em termos de custo, uma vez que o de custo uniforme conseguiu chegar a uma solução de custo menor, sendo esta ideal este problema.


Os resultados obtidos são consistentes com o esperado para estes algoritmos:
 - O de Profundidade-Primeiro entra num ciclo infinito, confirmando assim que o algoritmo não é completo;
 - O de Largura-Primeiro demora 0.00675s a encontrar uma solução com custo 9, confirmando que o algoritmo é completo, mas não optimal uma vez que é o algoritmo mais custoso. Seria de esperar uma vez que os custos não são homogéneos;
 - O de Aprofundamento Progressivo demorou 0.00762s a chegar a uma solução com custo 7, provando que o algoritmo é completo, no entanto não optimal, pelo mesmo motivo do de Largura-Primeiro, os custos não homogéneos. Verifica-se ainda que chegou a uma solução com menor custo que o de Largura-Primeiro;
 - O de Custo Uniforme chega a uma solução após 0.01504s de custo 6. Prova-se assim que o algoritmo é completo e optimal, apesar da maior complexidade temporal.

Dos resultados obtidos podemos concluir que o algoritmo de Profundidade-Primeiro entra num ciclo infinito, ficando ciclamente alternando entre duas casas.
Verifica-se que a complexidade temporal dos algoritmos de Largura-Primeiro e Aprofundamento Progressivo são parecidas ($\mathcal{O}(b^n)$), tendo apenas um desvio de +13.
Confirma-se ainda que o algoritmo de Custo Uniforme é completo e o optimal, apesar da maior complexidade temporal (desvio de +173).

Algumas considerações finais sobre a implementação do espaço de estado:
- Para esta implementação da classe EstadoPacman (Espaço de estados) a implementação dos métodos \_\_eq__, \_\_lf__ e \_\_hash__ afetam imenso a forma como os algoritmos efetuam a procura, um cálculo ideal da hash e uma comparação completa de todos os atributos da classe causam com que todos os estados gerados sejam únicos, e como o pacman pode-se mover para qualquer uma das casas, alguns algoritmos vão entrar em ciclo infinito (caso do de Profundidade-Primeiro).

### not sure about this part
Foi preciso encontrar a "melhor forma" de implementar estes métodos de forma a que todos os algoritmos de procura em grafo possam concluir a sua procura. Isto causa também com que alguns algoritmos não tenham comportamento consistente com o esperado para certas configurações do espaço de estados.

$\to $